In [10]:
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from cred import key
import pandas as pd
import numpy as np

In [83]:
def parse(data):
    """
    Parsing data to the same format as in "students-all.csv"
    
    returns: nicks array and dataframe
    """
    nicks = np.array(data['Nick'])
    mails = np.array(data['Feedback mail'])
    data = data.iloc[:, 2:-1]

    column_names = [
        'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob',
        'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures',
        'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'internet',
        'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences',
        'G1', 'G2', 'G3'
    ]

    data.columns = column_names

    # deafults
    data['school'] = np.repeat("GP", data.shape[0])
    data['major'] = np.repeat("mat", data.shape[0])
    data['higher'] = np.repeat("yes", data.shape[0])

    # corect order
    data = data[[
        'school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu',
        'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime',
        'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities',
        'nursery', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc',
        'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'major'
    ]]
    
    data['age'] = pd.to_numeric(data['age'])
    data['sex'] = np.where(data.sex == 'Male', 'M', 'F')
    data['address'] = np.where(data.address == 'Rural', 'R', 'U')
    data['famsize'] = np.where(data.famsize == 'more than 3', 'GT3', 'LE3')
    data['Pstatus'] = np.where(data.Pstatus == 'living apart', 'A', 'T')

    d1 = {
        'none': 0,
        'primary education': 1,
        'middle school': 2,
        'high school': 3,
        'higher education': 4
    }

    data['Medu'] = [d1[item] for item in list(data.Medu)]
    data['Fedu'] = [d1[item] for item in list(data.Fedu)]

    d2 = {
        'teacher': 'teacher',
        'healthcare': 'health',
        'civil services': 'civil',
        'home': 'at_home',
        'other': 'other'
    }

    data['Mjob'] = [d2[item] for item in list(data.Mjob)]
    data['Fjob'] = [d2[item] for item in list(data.Fjob)]

    d3 = {
        'close to home': 'home',
        'school reputation': 'reputation',
        'course preference': 'course',
        'other': 'other'
    }

    data['reason'] = [d3[item] for item in list(data.reason)]
    
    d4 = {'Mother':'mother', 'Father':'father', 'other':'other'}
    
    data['guardian'] = [d4[item] for item in list(data.guardian)]

    d5 = {'< 15':1, '15-30':2, '30-60':3,'60 >':4}
    
    data['traveltime'] = [d5[item] for item in list(data.traveltime)]
    
    d6 = {'< 2':1, '2-5':2, '5-10':3,'10 >':4}
    
    data['studytime'] = [d6[item] for item in list(data.studytime)]
    
    data['G1'] = np.round(pd.to_numeric(data.G1)/5)*20 
    data['G2'] = (pd.to_numeric(data.G2)/5)*20 
    data['G3'] = (pd.to_numeric(data.G3)/5)*20 
    
    data['absences'] = pd.to_numeric(data.absences) 
    
    
    return nicks, mails, data

In [18]:
service = build('sheets', 'v4',developerKey=key)
sheet = service.spreadsheets()

In [19]:
SAMPLE_SPREADSHEET_ID = '1e1tWLI0vD05bUj-wLWicOnl0iU-GWz0aaWEtRDlTQ2M'

cols = "AG"
rows = 4000

SAMPLE_RANGE_NAME = 'A1:'+ cols + str(rows)

In [20]:
SAMPLE_RANGE_NAME

'A1:AG4000'

In [61]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])
data = pd.DataFrame(values)
data

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,Sygnatura czasowa,Nick,Sex,Age,Adress,Family size,Parent status,Mother education,Father education,Mother job,...,Free time after school,Going out with friends,Workday alcohol consumption,Weekend alcohol consumption,Health,Number of school absences (0-90),3rd semester grade (2-5),4rd semester grade (2-5),Grade up to this date (2-5),Feedback mail
1,2020-04-18 22:36:19,testuś,Male,19,Rural,Less or equal than 3,living apart,middle school,middle school,civil services,...,3,2,4,2,5,13,3.14,4.19,3.90,None
2,2020-04-18 22:40:00,testuś2,Female,24,Urban,more than 3,living toogether,none,middle school,other,...,3,3,3,3,3,4,4,4,4,None
3,2020-04-19 10:49:35,test3,Male,21,Urban,more than 3,living toogether,primary education,none,healthcare,...,3,1,5,5,1,53,4,5,3,None
4,2020-04-19 12:09:57,test4,Male,20,Urban,more than 3,living toogether,middle school,primary education,teacher,...,3,3,3,3,3,32,4,3,2,None
5,2020-04-19 12:17:25,test_czy_submit_dziala,Male,20,Urban,more than 3,living apart,none,higher education,healthcare,...,3,3,3,3,3,34,2,3,4,None
6,2020-04-19 13:31:29,3,Male,20,Urban,more than 3,living apart,none,none,teacher,...,2,2,2,2,2,56,4,5,3,None
7,2020-04-19 15:00:03,silnoręki,Female,19,Urban,Less or equal than 3,living toogether,higher education,higher education,civil services,...,5,5,5,5,5,89,2,2,2,None
8,2020-04-19 17:20:55,Haha,Female,22,Urban,more than 3,living toogether,higher education,higher education,other,...,1,1,4,2,5,5,4,4,4,okk.olga@outlook.com


In [62]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])
data = pd.DataFrame(values)
data.columns = data.iloc[0,:]
data = data.iloc[1:,:]
data.head(10)

,Sygnatura czasowa,Nick,Sex,Age,Adress,Family size,Parent status,Mother education,Father education,Mother job,...,Free time after school,Going out with friends,Workday alcohol consumption,Weekend alcohol consumption,Health,Number of school absences (0-90),3rd semester grade (2-5),4rd semester grade (2-5),Grade up to this date (2-5),Feedback mail
1,2020-04-18 22:36:19,testuś,Male,19,Rural,Less or equal than 3,living apart,middle school,middle school,civil services,...,3,2,4,2,5,13,3.14,4.19,3.90,None
2,2020-04-18 22:40:00,testuś2,Female,24,Urban,more than 3,living toogether,none,middle school,other,...,3,3,3,3,3,4,4,4,4,None
3,2020-04-19 10:49:35,test3,Male,21,Urban,more than 3,living toogether,primary education,none,healthcare,...,3,1,5,5,1,53,4,5,3,None
4,2020-04-19 12:09:57,test4,Male,20,Urban,more than 3,living toogether,middle school,primary education,teacher,...,3,3,3,3,3,32,4,3,2,None
5,2020-04-19 12:17:25,test_czy_submit_dziala,Male,20,Urban,more than 3,living apart,none,higher education,healthcare,...,3,3,3,3,3,34,2,3,4,None
6,2020-04-19 13:31:29,3,Male,20,Urban,more than 3,living apart,none,none,teacher,...,2,2,2,2,2,56,4,5,3,None
7,2020-04-19 15:00:03,silnoręki,Female,19,Urban,Less or equal than 3,living toogether,higher education,higher education,civil services,...,5,5,5,5,5,89,2,2,2,None
8,2020-04-19 17:20:55,Haha,Female,22,Urban,more than 3,living toogether,higher education,higher education,other,...,1,1,4,2,5,5,4,4,4,okk.olga@outlook.com


In [84]:
n, m, df = parse(data)

/home/hania/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/hania/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


TypeError: cannot convert the series to <class 'int'>

In [78]:
df.describe()
df_dummies = pd.get_dummies(df, drop_first=True)
y_predict_prob = xgb_model.predict_proba(df_dummies)[:,1]

ValueError: feature_names mismatch: ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother', 'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes', 'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes', 'major_por'] ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'absences', 'G1', 'G2', 'G3', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T', 'Mjob_health', 'Mjob_other', 'Mjob_teacher', 'Fjob_health', 'Fjob_other', 'Fjob_teacher', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother', 'guardian_other', 'failures_2', 'failures_3', 'failures_4', 'schoolsup_Yes', 'famsup_Yes', 'paid_Yes', 'activities_Yes', 'nurse

In [81]:
X_test.dtypes

age                  int64
Medu                 int64
Fedu                 int64
traveltime           int64
studytime            int64
failures             int64
famrel               int64
freetime             int64
goout                int64
Dalc                 int64
Walc                 int64
health               int64
absences             int64
G1                   int64
G2                   int64
G3                   int64
school_MS            uint8
sex_M                uint8
address_U            uint8
famsize_LE3          uint8
Pstatus_T            uint8
Mjob_health          uint8
Mjob_other           uint8
Mjob_services        uint8
Mjob_teacher         uint8
Fjob_health          uint8
Fjob_other           uint8
Fjob_services        uint8
Fjob_teacher         uint8
reason_home          uint8
reason_other         uint8
reason_reputation    uint8
guardian_mother      uint8
guardian_other       uint8
schoolsup_yes        uint8
famsup_yes           uint8
paid_yes             uint8
a

In [82]:
df_dummies.dtypes

age                    int64
Medu                   int64
Fedu                   int64
traveltime             int64
studytime              int64
absences               int64
G1                   float64
G2                   float64
G3                   float64
sex_M                  uint8
address_U              uint8
famsize_LE3            uint8
Pstatus_T              uint8
Mjob_health            uint8
Mjob_other             uint8
Mjob_teacher           uint8
Fjob_health            uint8
Fjob_other             uint8
Fjob_teacher           uint8
reason_home            uint8
reason_other           uint8
reason_reputation      uint8
guardian_mother        uint8
guardian_other         uint8
failures_2             uint8
failures_3             uint8
failures_4             uint8
schoolsup_Yes          uint8
famsup_Yes             uint8
paid_Yes               uint8
activities_Yes         uint8
nursery_Yes            uint8
internet_Yes           uint8
famrel_3               uint8
famrel_4      

In [74]:
df.sex

1    M
2    F
3    M
4    M
5    M
6    M
7    F
8    F
Name: sex, dtype: object

In [45]:
from watcher import entries_processed

In [46]:
entries_processed

0

In [50]:
#service = build('sheets', 'v4',developerKey=key)
#sheet = service.spreadsheets()
def get_new_entries():
    global entries_processed
    cols = "AG"
    rows = 50

    SAMPLE_RANGE_NAME = 'A1:'+ cols + str(rows)
    
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])
    data = pd.DataFrame(values)
    data.columns = data.iloc[0,:]
    data = data.iloc[1+entries_processed:,:]
    
    n,m,df = parse(data)
    
    entries_processed += len(n)
    # should write to file just in case
    return n,m,df

In [58]:
n,m,df_gotten = get_new_entries()

In [52]:
entries_processed

8

In [54]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV

df = pd.read_csv("students-all.csv")
# remove rownames
df = df.iloc[:,1:]
df['romantic'] = np.where(df['romantic']=='yes', 1, 0)
df_dummies = pd.get_dummies(df, drop_first=True)


X_train, X_test, y_train, y_test = train_test_split(df_dummies.drop(['romantic'], axis=1),
                                                    df_dummies['romantic'], test_size = 0.2, random_state = 666)

xgb_model = xgb.XGBClassifier(max_depth = 7,
                              booster = "dart",
                              colsample_bytree = 0.3,
                              learning_rate = 0.39,
                              reg_alpha = 0.9,
                              reg_lambda = 1.8,
                              subsample = 0.8)
xgb_model.fit(X_train,y_train)

#y_prob = xgb_model.predict_proba(X_test)[:,1]

y_predict_prob = xgb_model.predict_proba(df)[:,1]

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields school, sex, address, famsize, Pstatus, Mjob, Fjob, reason, guardian, schoolsup, famsup, paid, activities, nursery, higher, internet, major

In [59]:
df_gotten.describe()

,age,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,G1,G2,G3
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
